In [ ]:
library(HMM)
library(mclust)
library(mvtnorm)

In [ ]:
dim(faithful)
head(faithful)

In [ ]:
mygauss <- function(data, mean, sd){
    p = nrow(sd)
    svd_obj = svd(sd)
    D = svd_obj$d
    U = svd_obj$u
    V = svd_obj$v
    Dt = diag(1/sqrt(D))
    Sinv = U %*% Dt^2 %*% t(U) 
    xt = Dt %*% t(U) %*% data
    mt = Dt %*% t(U) %*% mean
    a = t(xt-mt) %*% Sinv %*% (xt-mt)
    return (exp(-0.5 * a)/((2*pi)^p * (det(sd))^0.5))
}

Estep <- function(data, G, para){
    # Your Code
    prob <- para$prob
    mu <- para$mean
    Sigma <- para$Sigma
    loglik <- para$loglik
    n <- nrow(data)
    # need to come up with a way to evalue multivariate gaussian distrbutions,
    # but let's focus on getting the algorithm correct right now.
    # assume we have G means, G Sigmas, G pis
    p = prob * mygauss(data, mu, Sigma)
    sums = rowSums(p)
    p = p / sums
  
    # Return the n-by-G probability matrix
    return(prob)
  }

Mstep <- function(data, G, para, post.prob){ 
    # Your Code
    # Return the updated parameters
    para[1] = colSums(post.prob) / nrows(post.prob)
    mu = t(data) %*% post.prob
    mu_sum = colSums(mu)
    para[2] = mu / t(mu_sum)
    Sigma = post.prob * ((data - mu) %*% (data - mu))
    para[3] = colSums(Sigma) / colSums(post.prob)
    para[4] = loglik(data, G, para)
    
    return (para)
  }

loglik <- function(data, G, para){
    # compute loglikelihood
}

myEM <- function(data, itmax, G, para){
  # itmax: number of of iterations
  # G:     number of components
  # para:  list of (prob, mean, Sigma, loglik)
  d = as.matrix(data)
  for(t in 1:itmax){
    print(t)
    post.prob <- Estep(d, G, para)
    para <- Mstep(d, G, para, post.prob)
  }
  
  # update para$loglik   
  
  return(para)
}

In [ ]:
n <- nrow(faithful)
G <- 2
set.seed(7568)  # replace 234 by the last 4-dig of your University ID
gID <- sample(1:G, n, replace = TRUE)
Z <- matrix(0, n, G)
for(k in 1:G)
  Z[gID == k, k] <- 1 
ini0 <- mstep(modelName="EEE", faithful , Z)$parameters

In [ ]:
para0 <- list(prob = ini0$pro, 
              mean = ini0$mean, 
              Sigma = ini0$variance$Sigma, 
              loglik = NULL)

In [ ]:
myEM(d=faithful, itmax=20, G=K, para=para0)